In [1]:
#!pip install mysql-connector-python

In [2]:
# Import necessary libraries
import pandas as pd
import mysql.connector
import numpy as np

In [3]:
# MySQL database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'put mysql server connection password'
}

In [4]:
# Establish connection to MySQL database
conn = mysql.connector.connect(**db_config)

In [5]:
# Function to execute MySQL queries
def execute_query(query):
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()

In [6]:
# Create database if not exists
create_db_query = "CREATE DATABASE IF NOT EXISTS etl_example;"
execute_query(create_db_query)

In [7]:
# Use the database
use_db_query = "USE etl_example;"
execute_query(use_db_query)

In [8]:
# Create raw_data table
create_raw_data_table_query = """
CREATE TABLE IF NOT EXISTS raw_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    age INT,
    city VARCHAR(255)
);
"""
execute_query(create_raw_data_table_query)

In [12]:
import numpy as np

# Generate sample data with null values and inconsistencies
num_rows = 100
names = ['John', 'Jane', 'Mike', 'Emily', 'Chris', None]  # Including null values
ages = np.random.randint(18, 60, size=num_rows).astype(float)  # Convert to float to allow NaN
nan_indices = np.random.choice(range(num_rows), size=num_rows//10, replace=False)  # Select 10% indices for null values
ages[nan_indices] = np.nan  # Introducing some null values

# Define initial cities list
initial_cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', '']

# Calculate repetition factor to ensure the list covers num_rows
repetition_factor = num_rows // len(initial_cities) + 1

# Now, extend the list to match num_rows
cities = initial_cities * repetition_factor
np.random.shuffle(cities)  # Shuffle to introduce inconsistencies
cities = cities[:num_rows]  # Trim to match num_rows


In [13]:
# Create DataFrame
sample_df = pd.DataFrame({
    'name': np.random.choice(names, size=num_rows),
    'age': ages,
    'city': cities
})

In [14]:
sample_df.head()

,name,age,city
0,Emily,37.0,New York
1,Emily,20.0,Chicago
2,Chris,30.0,New York
3,Chris,56.0,
4,Chris,NaN,Chicago


In [15]:
len(sample_df)

100

In [16]:
# Insert sample data into raw_data table
for index, row in sample_df.iterrows():
    name_value = f"'{row['name']}'" if row['name'] is not None else 'NULL'
    age_value = str(row['age']) if not np.isnan(row['age']) else 'NULL'
    city_value = f"'{row['city']}'" if row['city'] != '' else 'NULL'
    
    insert_query = f"""
    INSERT INTO raw_data (name, age, city) 
    VALUES ({name_value}, {age_value}, {city_value})
    """
    execute_query(insert_query)


In [17]:
# Fetch data from raw_data table and display
fetch_query = "SELECT * FROM raw_data"
fetched_df = pd.read_sql_query(fetch_query, conn)

In [18]:
fetched_df.head()

,id,name,age,city
0,1,Emily,37.0,New York
1,2,Emily,20.0,Chicago
2,3,Chris,30.0,New York
3,4,Chris,56.0,None
4,5,Chris,NaN,Chicago


In [19]:
#check for missing values
(fetched_df.isna()).sum()

id       0
name    19
age     10
city    17
dtype: int64

In [20]:

fetched_df["city"].unique()

array(['New York', 'Chicago', None, 'Los Angeles', 'Phoenix', 'Houston'],
      dtype=object)

In [21]:
#summary statistics of continuos variables
fetched_df.describe()

,id,age
count,100.000000,90.000000
mean,50.500000,37.988889
std,29.011492,11.965770
min,1.000000,18.000000
25%,25.750000,28.000000
50%,50.500000,37.000000
75%,75.250000,48.000000
max,100.000000,59.000000


In [22]:
# Perform transformations using Pandas
# Cleaning data - removing rows with missing values
cleaned_df = fetched_df.dropna()

In [ ]:
# create a copy of our cleaned df to avoid SettingWithCopyWarning
cleaned_df = cleaned_df.copy()


In [32]:
#Remove rows with empty strings in 'city' column
cleaned_df = cleaned_df[cleaned_df['city'] != '']


In [28]:
#Apply transformations to 'name' column by changing to upper case
cleaned_df.loc[:, 'name'] = cleaned_df.loc[:, 'name'].str.upper()


In [29]:
cleaned_df.head()

,id,name,age,city
0,1,EMILY,37.0,New York
1,2,EMILY,20.0,Chicago
2,3,CHRIS,30.0,New York
5,6,JANE,42.0,New York
10,11,JOHN,48.0,Phoenix


In [30]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      56 non-null     int64  
 1   name    56 non-null     object 
 2   age     56 non-null     float64
 3   city    56 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 2.2+ KB


In [31]:
# Convert 'age' column to integer type
cleaned_df['age'] = cleaned_df['age'].astype(int)

In [34]:
# Create a new table for cleaned data in MySQL database
create_cleaned_table_query = """
CREATE TABLE IF NOT EXISTS cleaned_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    age INT,
    city VARCHAR(255)
);
"""
execute_query(create_cleaned_table_query)

In [35]:
# Insert cleaned data into MySQL table
for index, row in cleaned_df.iterrows():
    insert_query = f"""
    INSERT INTO cleaned_data (name, age, city) 
    VALUES ('{row['name']}', {row['age']}, '{row['city']}')
    """
    execute_query(insert_query)

In [36]:
# Fetch data from cleaned table and display
fetch_cleaned_query = "SELECT name, age, city FROM cleaned_data"
cleaned_df_fetched = pd.read_sql_query(fetch_cleaned_query, conn)
print(cleaned_df_fetched.head())

    name  age      city
0  EMILY   37  New York
1  EMILY   20   Chicago
2  CHRIS   30  New York
3   JANE   42  New York
4   JOHN   48   Phoenix
